In [1]:
from bindsnet.network import Network
from bindsnet.network.nodes import Input, LIFNodes
from bindsnet.network.topology import Connection
import lab_utils
import numpy as np
import torch
import matplotlib.pyplot as plt
from bindsnet.network import Network
from bindsnet.network.nodes import RealInput, LIFNodes
from bindsnet.network.topology import Connection
from bindsnet.network.monitors import Monitor
from bindsnet.analysis.plotting import plot_spikes, plot_voltages

In [2]:
unique, counts = np.unique(raw_data[:, -1], return_counts=True)
labels_list = unique
index_to_labels_dict = {index: value for index, value in enumerate(unique[1:])}
label_to_index_dict = {value: index for index, value in index_to_labels_dict.items()}
n_labels = len(labels_list)
print('N labels = ', n_labels)
print('Labels: ', labels_list)
print('Label to index dict ', label_to_index_dict)

gesture_samples = (raw_data[:, -1] != -1).ravel()
gesture_switch_indices = np.argwhere(np.diff(gesture_samples) != 0).ravel() + 1 
gesture_switch_indices = np.insert(gesture_switch_indices, 0, 0)
print('Indices where gestures were switched: \n', gesture_switch_indices)

subject_gestures_data = []
subject_gestures_labels = []
for i in range(len(gesture_switch_indices) - 1):
    subject_gestures_data.append(
        raw_data[gesture_switch_indices[i]:gesture_switch_indices[i + 1], :-1])
    subject_gestures_labels.append(raw_data[gesture_switch_indices[i], -1])
subject_gestures_labels = np.asarray(subject_gestures_labels)
subject_gestures_data = np.asarray(subject_gestures_data)

gesture_indices_by_id = np.argwhere(subject_gestures_labels != -1).ravel()
subject_gestures_data = subject_gestures_data[gesture_indices_by_id]
subject_gestures_labels = subject_gestures_labels[gesture_indices_by_id]

for i, gesture_data in enumerate(subject_gestures_data):
    print(subject_gestures_labels[i], ' ', gesture_data.shape)
    
def split_data_in_epochs(subject_gestures_data, subject_gestures_labels, epoch_length, shift_length):
    sequence_data = []
    for i, label in enumerate(subject_gestures_labels):

        for data_index in range(0, len(subject_gestures_data[i]) - epoch_length, shift_length):
            sequence = np.asarray(subject_gestures_data[i][data_index:data_index + epoch_length, :])
            sequence_data.append([sequence, label])
    return np.asarray(sequence_data)

epochs_data = split_data_in_epochs(subject_gestures_data, subject_gestures_labels, 200, 50)
#Undo shuffling
#np.random.shuffle(epochs_data)
X = np.stack(epochs_data[:, 0], axis=0)
# softmax accepts only sequential labels starting from 0
y = np.asarray([label_to_index_dict[label] for label in epochs_data[:, 1]])
X.shape

y_unique, y_counts = np.unique(y, return_counts=True)
print("Labels balance:\n\n", np.asarray((y_unique, y_counts)).T)
min_label_n = np.min(y_counts)
print('Min size of label = ', min_label_n)
print(X.shape)

NameError: name 'raw_data' is not defined

In [ ]:
sampling_rate = 650
# Specify path to data file that needs to be analysed
file_path = '/home/maria/Documents/emg_data/new_data/1-8_fist_spread.emg'
#file_path='/home/maria/Documents/emg_data/new_data/1-5_fingers_6_relax.emg'
raw_data = lab_utils.read_data(file_path)
lab_utils.plot_raw_data(raw_data)
lab_utils.plot_labels(raw_data)

n_channels=8
print(raw_data.shape)

In [ ]:
# Simulation time.
time = 200.0

# Create the network.
network = Network()

# Create two populations of neurons, one to act as the "source"
# population, and the other, the "target population".
source_layer = RealInput(n=8)
target_layer = LIFNodes(n=10)

network.add_layer(
    layer=source_layer, name="A"
)
network.add_layer(
    layer=target_layer, name="B"
)

print(X.shape)
from scipy.stats import zscore
input_data=X[0,:,:].reshape(200,8)
input_data=zscore(input_data,axis=0).astype('float32')+1
#input_data = torch.bernoulli(0.1 * torch.ones(time, source_layer.n)).byte()
print(input_data.shape)
source_monitor = Monitor(
    obj=source_layer,
    state_vars=("s",),  # Record spikes and voltages.
    time=time,  # Length of simulation (if known ahead of time).
)
target_monitor = Monitor(
    obj=target_layer,
    state_vars=("s", "v"),  # Record spikes and voltages.
    time=time,  # Length of simulation (if known ahead of time).
)

forward_connection = Connection(
    source=source_layer,
    target=target_layer,
    w=0.05 + 0.1 * torch.randn(source_layer.n, target_layer.n),  # Normal(0.05, 0.01) weights.
)

network.add_connection(
    connection=forward_connection, source="A", target="B"
)

network.add_monitor(monitor=source_monitor, name="A")
network.add_monitor(monitor=target_monitor, name="B")
inputs = {"A": torch.Tensor(input_data)}


#print(inputs.type)
#print(input_data.type)

network.run(inpts=inputs, time=time)
spikes = {
    "A": source_monitor.get("s"), "B": target_monitor.get("s")
}
voltages = {"B": target_monitor.get("v")}

plt.ioff()
plot_spikes(spikes)
plot_voltages(voltages, plot_type="line")
plt.show()


